# Self-driving car Nanodegree - Term 1

## Project 2: **Behavior Cloning** 
***
In this project, We will develop an algorithm pipeline to detect lane lines in images. In addition to implementing code, there is a brief [writeup](https://github.com/tranlyvu/autonomous-vehicle-projects/blob/master/Finding%20Lane%20Lines/writeup.md)

#### Importing packages

In [39]:
import csv
from keras.models import Sequential
from keras.layers import Flatten, Dense, Convolution2D, Cropping2D,Lambda
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


#### Defining helper functions

In [6]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1:
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset + batch_size]
            images = []
            angles = []
            for batch_sample in batch_samples:
                name = '../../data/IMG/' + batch_image[0].split('/')[-1]
                center_name = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            x_train = np.array(images)
            y_train = np.array(angels)
            yield sklean.utils.shuffle(x_train, y_train)

#### Loading the data

In [30]:
'''Read data'''
rows = []
with open('../../../data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        rows.append(row)

        
images = []
measurements = []
for line in rows[1:]:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = '../../../data/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)

[[[208 163 130]
  [208 163 130]
  [208 163 130]
  ..., 
  [210 163 131]
  [210 163 131]
  [210 163 131]]

 [[208 163 130]
  [208 163 130]
  [208 163 130]
  ..., 
  [210 163 131]
  [210 163 131]
  [210 163 131]]

 [[209 164 131]
  [209 164 131]
  [209 164 131]
  ..., 
  [211 164 132]
  [211 164 132]
  [211 164 132]]

 ..., 
 [[ 83  99 112]
  [ 88 104 117]
  [ 89 106 119]
  ..., 
  [ 99 116 135]
  [106 123 142]
  [111 128 147]]

 [[ 92 109 122]
  [ 88 105 118]
  [ 82  99 112]
  ..., 
  [104 123 144]
  [ 96 115 136]
  [ 91 110 131]]

 [[ 87 104 117]
  [ 79  96 109]
  [ 79  95 111]
  ..., 
  [115 134 155]
  [106 125 146]
  [ 99 118 139]]]
0.0
[[[208 163 130]
  [208 163 130]
  [208 163 130]
  ..., 
  [210 163 131]
  [210 163 131]
  [210 163 131]]

 [[208 163 130]
  [208 163 130]
  [208 163 130]
  ..., 
  [210 163 131]
  [210 163 131]
  [210 163 131]]

 [[209 164 131]
  [209 164 131]
  [209 164 131]
  ..., 
  [211 164 132]
  [211 164 132]
  [211 164 132]]

 ..., 
 [[156 174 197]
  [167 185 2

#### Pre-processing data 

    a. Data augmentation: Flipping images
    b. Normalization
    c. Mean centering


In [44]:
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    image_flipped = np.fliplr(image)
    measurement_flipped = measurement * (-1)
    augmented_images.append(image)
    augmented_images.append(image_flipped)
    augmented_measurements.append(measurement)
    augmented_measurements.append(measurement_flipped)

X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

model = Sequential()
#The cameras in the simulator capture 160 pixel by 320 pixel images., after cropping, it is 90x320
model.add(Cropping2D(cropping = ((50,20), (0,0))))
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(90, 320, 3)))


#### Defining model artchitecture

In first attempt, I used 9-layers network from [end to end learning for self-driving cars](https://arxiv.org/pdf/1604.07316.pdf) by NVIDIA

In [45]:
model.add(Convolution2D(24, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(36, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(48, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))      
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

In [47]:
'''Training: using MSE for regression'''
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split = 0.2, shuffle = True, verbose = 1)

model.save('model.h5')


train_samples, validation_samples = train_test_split(rows, test_size = 0.2)

train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

history_object = model.fit_generator(train_generator,
                                    sample_per_epoch = len(train_samples),
                                    validation_data = validation_generator,
                                    nb_val_Samples = len(validation_examples),
                                    nb_epoch=5)
print(history_object.history.key())

###plot the training and validation loss for each epoch
plt.plot(history(history_objet.history['loss']))
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

Train on 12857 samples, validate on 3215 samples
Epoch 1/10
12857/12857 [==============================] - 317s - loss: 0.0199 - val_loss: 0.0167
Epoch 2/10
12857/12857 [==============================] - 318s - loss: 0.0167 - val_loss: 0.0167
Epoch 3/10
 4928/12857 [==========>...................] - ETA: 177s - loss: 0.0155

KeyboardInterrupt: 